# Project Report for CS598 DL4H in Spring 2023

{anevala2, yutaron2}@illinois.edu

- Group ID: 64
- Paper ID: 10

## Summary of the report and findings

A summary of the report and findings (Reproducibility Summary, about 200 words)

## Overview of the MIMIC Ⅲ dataset

An overview of the data with any helpful charts and visualizations from the report and ideally directly using the dataset in the notebook (maybe link to the data folder or URL).

We use three CSV files from the MIMIC Ⅲ dataset.

- ADMISSIONS.csv
- DIAGNOSES_ICD.csv
- ccs_multi_dx_tool_2015.csv

We preprocessed these these files to generate 
- `output.hfs`: contains the heart failure label (0: normal, 1: heart failure) for each patient
- `output2.seqs`: contains a list of visit (in ICD9 codes) for each patient
- `output2.types`: contains the map from ICD9 codes to ICD-9 labels

In [33]:
import pickle

with open('./output/output.hfs', 'rb') as f:
	hfs = pickle.load(f)
with open('./output2/output2.seqs', 'rb') as f:
	seqs = pickle.load(f)
with open('./output2/output2.types', 'rb') as f:
	types = pickle.load(f)

print("number of patients:", len(hfs))
print("number of heart failure patients:", sum(hfs))
print("ratio of heart failure patients: %10.2f" % (float(sum(hfs)) / len(hfs)))
print("number of unique ICD9 codes:", len(types))

('number of patients:', 5447)
('number of heart failure patients:', 1280)
ratio of heart failure patients:       0.23
('number of unique ICD9 codes:', 4512)


Also, we prepared ancestor information from medical ontologies data to produce:

- output2.level1.pk
- output2.level2.pk
- output2.level3.pk
- output2.level4.pk
- output2.level5.pk

## Overview of methodology and experiments

An overview of the methodology and experiments run,ideally with executable code examples

We implemented the GRAM model for heart faiure prediction in `gram_hf.py`. The following code imported it as `gram_hf` and train the model with 5 epochs.

In [36]:
import numpy as np
import gram_hf

if __name__ == '__main__':
	seq_file = 'output2/output2.seqs'
	tree_file = 'output2/output2'
	label_file = 'output/output.hfs'
	out_file = 'ipynb/gram_hf'
	inputDimSize = gram_hf.calculate_dimSize(seq_file)
	numClass = 1 # predicte a binary value (heart failure)
	numAncestors = gram_hf.get_rootCode(tree_file+'.level2.pk') - inputDimSize + 1

	gram_hf.train_GRAM(
		seqFile=seq_file,
		inputDimSize=inputDimSize,
		treeFile=tree_file,
		numAncestors=numAncestors,
		labelFile=label_file,
		numClass=numClass,
		outFile=out_file,
		embFile='',
		embDimSize=128,
		hiddenDimSize=128,
		attentionDimSize=128,
		batchSize=100,
		max_epochs=5,
		L2=0.001,
		dropoutRate=0.5,
		logEps=1e-8,
		train_ratio=0.7
	)

 Building the model ...  done!!
Constructing the optimizer ...  done!!
Loading data ...  done!!
Optimization start !!
Epoch:0, Duration:12.177816, Train_Cost:0.647768, Valid_Cost:0.536243, Test_Cost:0.493310
Epoch:1, Duration:12.339224, Train_Cost:0.570731, Valid_Cost:0.518706, Test_Cost:0.486453
Epoch:2, Duration:12.307639, Train_Cost:0.534188, Valid_Cost:0.498042, Test_Cost:0.445167
Epoch:3, Duration:12.324813, Train_Cost:0.511874, Valid_Cost:0.485034, Test_Cost:0.437686
Epoch:4, Duration:16.401006, Train_Cost:0.491118, Valid_Cost:0.465700, Test_Cost:0.432930
Best Epoch:4, Avg_Duration:13.110100, Train_Cost:0.491118, Valid_Cost:0.465700, Test_Cost:0.432930


We also implemented the GRAM model without the attention mechanism in `gram_hf_no_attention.py`. The following code imported it as `gram_hf_no_attention` and train the model with 5 epochs.

In [39]:
import gram_hf_no_attention as gram_hf

if __name__ == '__main__':
	seq_file = 'output2/output2.seqs'
	tree_file = 'output2/output2'
	label_file = 'output/output.hfs'
	out_file = 'ipynb/gram_hf_no_attention' # for no attention
	inputDimSize = gram_hf.calculate_dimSize(seq_file)
	numClass = 1 # predicte a binary value (heart failure)
	numAncestors = gram_hf.get_rootCode(tree_file+'.level2.pk') - inputDimSize + 1

	gram_hf.train_GRAM(
		seqFile=seq_file,
		inputDimSize=inputDimSize,
		treeFile=tree_file,
		numAncestors=numAncestors,
		labelFile=label_file,
		numClass=numClass,
		outFile=out_file,
		embFile='',
		embDimSize=128,
		hiddenDimSize=128,
		attentionDimSize=128,
		batchSize=100,
		max_epochs=5,
		L2=0.001,
		dropoutRate=0.5,
		logEps=1e-8,
		train_ratio=0.7
	)

Building the model ...  done!!
Constructing the optimizer ...  done!!
Loading data ...  done!!
Optimization start !!
Epoch:0, Duration:2.722397, Train_Cost:0.516371, Valid_Cost:0.576333, Test_Cost:0.501965
Epoch:1, Duration:2.729736, Train_Cost:0.479058, Valid_Cost:0.493964, Test_Cost:0.458061
Epoch:2, Duration:2.700339, Train_Cost:0.441143, Valid_Cost:0.483906, Test_Cost:0.435884
Epoch:3, Duration:2.674544, Train_Cost:0.399495, Valid_Cost:0.518985, Test_Cost:0.508718
Epoch:4, Duration:2.727133, Train_Cost:0.379071, Valid_Cost:0.391843, Test_Cost:0.395522
Best Epoch:4, Avg_Duration:2.710830, Train_Cost:0.379071, Valid_Cost:0.391843, Test_Cost:0.395522


We calculated AUC for each model as follows.

In [47]:
from sklearn.metrics import roc_auc_score
import pickle

with open('./ipynb/gram_hf_0.7.test_set', 'rb') as f:
    test_data = pickle.load(f)
with open('./ipynb/gram_hf_0.7_4.test_probs', 'rb') as f:
    y_pred_probs = pickle.load(f)

y_true_labels = test_data[1]
auc = roc_auc_score(y_true_labels, y_pred_probs)

print("GRAM with attention AUC:", auc)

('GRAM with attention AUC:', 0.8514355428251791)


In [48]:
from sklearn.metrics import roc_auc_score
import pickle

with open('./ipynb/gram_hf_no_attention_0.7.test_set', 'rb') as f:
    test_data = pickle.load(f)
with open('./ipynb/gram_hf_no_attention_0.7_4.test_probs', 'rb') as f:
    y_pred_probs = pickle.load(f)

y_true_labels = test_data[1]
auc = roc_auc_score(y_true_labels, y_pred_probs)

print("GRAM without attention AUC:", auc)

('GRAM without attention AUC:', 0.8606023782657755)


## Summary of the key results

## References

- Edward Choi, Mohammad Taha Bahadori, Le Song, Walter F Stewart, and Jimeng Sun. 2017. Gram: graph-based attention model for healthcare represen-tation learning. In Proceedings of the 23rd ACM SIGKDD international conference on knowledge discovery and data mining, pages 787–795.
- Jeffrey Pennington, Richard Socher, and Christopher D Manning. 2014. Glove: Global vectors for word representation. In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP), pages 1532–1543.